# **Installing all of the packages**

In [ ]:
!pip install -U accelerate bitsandbytes peft transformers datasets trl git-lfs wandb flash-attn sql-metadata scipy

In [3]:
import os
import torch
import pandas as pd
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, TaskType
from datasets import load_dataset
from sql_metadata import Parser
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from tqdm import tqdm

# **Connect to wandb**

In [4]:
os.environ["WANDB_PROJECT"]="deepseek_full_finetuning"

# **Loading the model**

In [5]:
model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant = True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #quantization_config=bnb_config,
    attn_implementation="flash_attention_2",
    torch_dtype = torch.bfloat16,
    device_map='auto',
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [ ]:
print(model)

# **Loading the dataset**

In [9]:
data_files = {"train": "full_finetuning_dataset.csv", "validation": "validation_dataset_formatted.csv"}
dataset = load_dataset('csv', data_files=data_files)

**Filtering rows with max tokens**

In [10]:
def formatting_prompts_func(training_dataset):
  output_texts = []
  for i in range(len(training_dataset['question'])):
    question = training_dataset['question'][i]
    correct_tables = training_dataset['correct_tables'][i]
    correct_columns = training_dataset['correct_columns'][i]
    database_schema = training_dataset['database_schema'][i]
    if correct_columns:
        correct_columns = ", ".join(set(correct_columns.split(", ")))
    correct_tables = ", ".join(set(correct_tables.split(", ")))
    user_message = f"""Given the following SQL tables, your job is to determine the columns and tables that the question is referring to.
{database_schema}
###
Question: {question}
"""
    assitant_message = f"""
```SQL
-- Columns: {correct_columns}
-- Tables: {correct_tables} ;
```
"""
    messages = [
    {"role": "user", "content": user_message},
    {"role": "assistant", "content": assitant_message},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    output_texts.append(text)
  return output_texts

In [11]:
response_template = "### Response:" #deepseek
# response_template = "[/INST]" #mistral"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# **Training Config**

In [25]:
lora_r = 64
lora_alpha = 32
lora_dropout = 0.1
output_dir = "./SFT"
num_train_epochs = 3
bf16 = True
overwrite_output_dir = True
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 16
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 5e-5
weight_decay = 0.01
lr_scheduler_type = "cosine"
warmup_ratio = 0.01
max_grad_norm = 0.3
group_by_length = True
auto_find_batch_size = False
save_steps = 50
logging_steps = 50
load_best_model_at_end= False
packing = False
save_total_limit=3
neftune_noise_alpha=5
report_to="wandb"
max_seq_length = 2100 #set based on the maximum number of tokens

In [26]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "v_proj",
        "k_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head"
    ],
    task_type=TaskType.CAUSAL_LM,
)

In [27]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
    evaluation_strategy=evaluation_strategy,
    max_grad_norm = max_grad_norm,
    auto_find_batch_size = auto_find_batch_size,
    save_total_limit = save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to=report_to,
    neftune_noise_alpha= neftune_noise_alpha
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=max_seq_length,
    packing=packing
)

In [ ]:
trainer.train()

In [ ]:
output_dir = os.path.join("./", "final_checkpoint")
trainer.model.save_pretrained(output_dir)